In [1]:
pip install google-api-python-client


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install google-auth-oauthlib


Note: you may need to restart the kernel to use updated packages.


In [7]:
import os
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.http import MediaFileUpload  # Add this import for MediaFileUpload

# Rest of your code remains the same...

# Path to the credentials JSON file downloaded from Google Cloud Console
credentials_path = 'credentials.json'

# Scopes for accessing Google Drive
SCOPES = ['https://www.googleapis.com/auth/drive']

# Authenticate and authorize the API
creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json')
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(credentials_path, SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

# Create Google Drive service
drive_service = build('drive', 'v3', credentials=creds)

# List of member names
member_names = [
    "Member1",
    "Member2",
    #...
    "Member3"
]

# Folder ID of the parent folder in Google Drive where you want to create member folders
parent_folder_id = '1EsXkZa_Y4Rz9LutN92XNQM-PScHthBnW'

for index, member_name in enumerate(member_names, start=1):
    # Create a folder for the member
    folder_metadata = {
        'name': member_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [parent_folder_id]
    }
    folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
    folder_id = folder.get('id')

    # Upload the respective evaluation sheet to the member's folder
    file_metadata = {
        'name': f'Evaluation_{index}_{member_name}.xlsx',
        'parents': [folder_id]
    }
    file_path = f'Result/Evaluation_{index}_{member_name}.xlsx'
    media = MediaFileUpload(file_path, mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

    print(f"Generated sheet for {member_name}. File uploaded to Google Drive.")


FileNotFoundError: [Errno 2] No such file or directory: 'Result/Evaluation_1_Member1.xlsx'